In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import ipywidgets as widgets
from ipywidgets import interact

# ESG APP

In [2]:
try:
    web_data = requests.get('https://finance.yahoo.com/quote/MSFT/sustainability?p=MSFT').text
    print('Loading data... Success!')
except:
    print('Could not get URL')

Success


- Select from the dropdown menu

In [3]:
soup = BeautifulSoup(web_data, 'html.parser')

In [4]:
esg_score = soup.find('div', {'class':'Fz(36px) Fw(600) D(ib) Mend(5px)'})
datapoint=esg_score.text
#print(datapoint)

In [5]:
scores = soup.find_all('div', {'class': 'D(ib) Fz(23px) smartphone_Fz(22px) Fw(600)'})
elements=[score.text for score in scores]

In [12]:
#elements

['0.4', '9.9', '5.2']

In [6]:
controversey_score = soup.find('div',{'class': 'D(ib) Fz(36px) Fw(500)'})
controversey_datapoint = controversey_score.text

In [7]:
df = pd.DataFrame({'Total ESG Score':datapoint,
                  'Environment Score':elements[0],
                  'Social Score':elements[1],
                   'Governance Score':elements[2],
                   'Controversy Score': controversey_datapoint},
                 index=['MSFT'])

#df

In [8]:
def get_tickers():
    wiki_page = requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies').text
    sp_data = pd.read_html(wiki_page)
    ticker_df = sp_data[0]
    ticker_options = ticker_df['Symbol']
    return ticker_options

In [9]:
def web_scraper(ticker):
    
    try:
        web_data = requests.get('https://finance.yahoo.com/quote/'+ticker+'/sustainability?p='+'ticker').text
    except:
        print('Could not get URL')
        
    soup = BeautifulSoup(web_data, 'html.parser')
    esg_score = soup.find('div', {'class':'Fz(36px) Fw(600) D(ib) Mend(5px)'})
    datapoint=esg_score.text
    scores = soup.find_all('div', {'class': 'D(ib) Fz(23px) smartphone_Fz(22px) Fw(600)'})
    elements=[score.text for score in scores]
    controversey_score = soup.find('div',{'class': 'D(ib) Fz(36px) Fw(500)'})
    controversey_datapoint = controversey_score.text
    
    df = pd.DataFrame({'Total ESG Score':datapoint,
                  'Environment Score':elements[0],
                  'Social Score':elements[1],
                   'Governance Score':elements[2],
                   'Controversy Score': controversey_datapoint},
                 index=[ticker])
    df = df.astype('float')
    df['Controversy Assessment'] = df.apply(lambda x: level(x['Controversy Score']), axis=1)
    return df

In [10]:
def level(x):
    if x==0.0:
        return 'No Controversy'
    elif x==1.0:
        return 'Little Controversy'
    elif x==2.0:
        return 'Little Controversy'
    elif x==3.0:
        return 'Relatively High Controversy'
    else:
        return 'Severe Controversy'

In [11]:
@interact
def get_esg(ticker=widgets.Dropdown(description='Ticker',
                                   options=get_tickers(),
                                   value='F')):
    esg_data = web_scraper(ticker)
    return esg_data

interactive(children=(Dropdown(description='Ticker', index=199, options=('MMM', 'ABT', 'ABBV', 'ABMD', 'ACN', …